# Tuning SGD Optimizer for Model Performance

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

In [7]:
import matplotlib
matplotlib.use('Agg')
%matplotlib inline

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import load_model
#https://github.com/jrosebr1/imutils
from imutils import paths
import imutils
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os

In [3]:
class AlexNet:
    #optimizer to use should be 'sgd'
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        #input shape is 227x227x3
        inputShape = (height, width, depth)
        model.add(Conv2D(filters=96, kernel_size=11, strides=4,
                         input_shape=inputShape, activation='relu'))
        model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
        model.add(Conv2D(filters=256, kernel_size=5, strides=1,
                         padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
        model.add(Conv2D(filters=384, kernel_size=3, strides=1,
                         padding='same', activation='relu'))
        model.add(Conv2D(filters=256, kernel_size=3, strides=1,
                         padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
        model.add(Flatten())
        model.add(Dense(4096, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(4096, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(classes, activation='softmax'))
        
        return model

In [4]:
def pre_process(width, height, path):
    """
    Resize and rescale images stored in image folder.
    
    Return pre-processed data and labels for the classes based
    on sub-directories in the image folder
    """
    #containers for pre-processed image data and class labels
    data = []
    labels = []

    #images directory contains 3 sub-directories: 'poison_ivy', 'poison_oak', 'poison_sumac'
    #randomly get image paths and shuffle them
    # current path 'C:\\Users\\jltsa\\Desktop\\Project_2\\images'
    image_paths = sorted(list(paths.list_images(path)))
    random.seed(42)
    random.shuffle(image_paths)

    #preprocess images to width x height pixels as required for LeNet
    for image_path in image_paths:
        image = cv2.imread(image_path)
        if image is not None:
            image = cv2.resize(image, (width, height))
            image = img_to_array(image)
            data.append(image)
    
            #Extract class labels
            label = image_path.split(os.path.sep)[-2]
            if label == 'poison_ivy':
                label = 0
            elif label == 'poison_oak':
                label = 1
            else:
                label = 2
            labels.append(label)
       
    #Scal pixel intensities from 0 to 1
    data = np.array(data, dtype='float') / 255.0
    labels = np.array(labels)
    
    return data, labels

In [23]:
def plot_acc_loss(epochs, model, title=None, save=False, path=None, save_as=None):
    """
    plot the accuracy and loss of the train and test data
    save plot optional
    """
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0,epochs), model.history['loss'], label='Train_loss')
    plt.plot(np.arange(0,epochs), model.history['val_loss'], label='Val_loss')
    plt.plot(np.arange(0,epochs), model.history['acc'], label='Train_acc')
    plt.plot(np.arange(0,epochs), model.history['val_acc'], label='Val_acc')
    plt.title('Training Loss and Accuracy' + ' ' + str(title))
    plt.xlabel('Epoch')
    plt.ylabel('Loss/Acc')
    plt.legend(loc='upper left')
    
    if save == True:
        plt.savefig(str(path)+'\\' + str(save_as) + '.png')

In [24]:
def plot_metric(epochs, metric, y_label, title=None, save=False, path=None, save_as=None):
    """
    plot single metric
    save plot optional
    """
    plt.style.use('ggplot')
    plt.figure()
    for met in range(len(metric)):
        plt.plot(np.arange(0,epochs), metric[met], label=y_label+str(met))
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel(y_label)
    plt.legend(loc='upper left')
    
    if save == True:
        plt.savefig(str(path)+'\\' + str(save_as) + '.png')
    

In [5]:
#To increase the amount of training data, build an image generator using data augmentation
aug_gen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                            height_shift_range=0.1, shear_range=0.2,
                            zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

In [27]:
#Initialize global training variables

EPOCHS = 2
#Learning rate
#LR = 1e-3
#Batch Size
BS = 15
path_to_img = 'C:\\Users\\jltsa\\Desktop\\Project_2\\images'
path_to_models = 'models\\'

In [26]:
sgd1 = SGD(lr=0.01, momentum=0.5, nesterov=True)
sgd2 = SGD(lr=0.01, momentum=0.8, nesterov=True)
sgd3 = SGD(lr=0.001, momentum=0.5, nesterov=True)
sgd4 = SGD(lr=0.001, momentum=0.8, nesterov=True)
sgd5 = SGD(lr=0.0001, momentum=0.5, nesterov=True)
sgd6 = SGD(lr=0.0001, momentum=0.8, nesterov=True)

opts = [sgd1, sgd2, sgd3, sgd4, sgd5, sgd6]

In [ ]:
def train_model(im_path, model_path, optimizers):
    loss = []
    acc = []
    val_loss = []
    val_acc = []
    
    data, labels = pre_process(227, 227, im_path)
    #Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)
    #One hot encoding
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    callbacks_list=[checkpoint, earlystop]
    
    for opt in range(len(optimizers)):
        #create a checkpoint to store the best weights of the model
        #to use these weights later, initialize the same model architecture that the weights were trained from
        #then call model.load_weights('best_weights_alex.h5')
        #can make predictions model.predict_classes(data)
        model_path += model_path+"_test"+str(opt)
        checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True)
        #add early stopping if 
        earlystop = EarlyStopping(monitor='val_loss', patience=3)
        
        
        #Initialize model
        model = AlexNet.build(width=227, height=227, depth=3, classes=3)
        #opt_alex = Adam(lr=LR, decay=LR / EPOCHS)

        #if model has 2 classes use loss='binary_crossentropy'
        #AlexNet performs better with 'sgd' optimizer
        model.compile(loss='categorical_crossentropy', optimizer=optimizers[opt], metrics=['accuracy'])

        #fit model using a generator to increase variation and amount of data
        model.fit_generator(aug_gen.flow(X_train, y_train, batch_size=BS),
                       validation_data=(X_test, y_test), steps_per_epoch=len(X_train // BS),
                       epochs=EPOCHS, callbacks=callbacks_list, verbose=1)
        #save history for loss and val scores
        loss.append(model.history['loss'])
        val_loss.append(model.history['val_loss'])
        acc.append(model.history['acc'])
        val_acc.append(model.history['val_acc'])

        #title = "AlexNet Using SGD"+str(opt)
        title = "test"+str(opt)
        plot_acc_loss(EPOCHS, model, title=title, save=True, path='plots', save_as="test"+str(opt))
 
    return loss, acc, val_loss, val_acc

AlexNet Using SGD0
AlexNet Using SGD1
AlexNet Using SGD2
AlexNet Using SGD3
AlexNet Using SGD4
AlexNet Using SGD5


In [ ]:
plot_metric(epochs, metric, 'Accuracy', title="Accuracies", save=False, path='plots', save_as="SGD_accs")